In [2]:
pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 5.7 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTENC
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, make_scorer
# import sklearn.metrics as skm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score, GridSearchCV
import pickle
import optuna
from xgboost import XGBClassifier

warnings.filterwarnings('ignore')

In [4]:
# Import the files from data_preprocessing.
X_train_oversampled = pd.read_csv("storage_files/X_train_oversampled.csv")
X_validate_transformed = pd.read_csv("storage_files/X_validate_transformed.csv")
X_test_transformed = pd.read_csv("storage_files/X_test_transformed.csv")
y_train_oversampled = pd.read_csv("storage_files/y_train_oversampled.csv")
y_validate = pd.read_csv("storage_files/y_validate.csv")
y_test = pd.read_csv("storage_files/y_test.csv")
importance_table = pd.read_csv("storage_files/importance_table.csv")

In [5]:
f = open('storage_files/max_no_features.txt','r')
max_no_features = int(f.readline())
f.close()
max_features = list(importance_table[:max_no_features]['Feature'])
X_train_oversampled = X_train_oversampled[max_features]
X_validate_transformed = X_validate_transformed[max_features]
X_test_transformed = X_test_transformed[max_features]

In [6]:
# Logistic Regression
print('\nLogistic Regression started.')
def objective_function(trial):
    C = trial.suggest_float('C', 0.1, 10, log=True)
    penalty = trial.suggest_categorical('penalty', ['l1', 'l2'])

    model = LogisticRegression(
        C=C,
        penalty=penalty,
        solver='liblinear',
        n_jobs=-1
    )
    metrics = ['roc_auc','accuracy','precison','recall','f1-score']
    # Using cross_val_score to get the average precision score for each fold
    scores = cross_val_score(model, X_train_oversampled, y_train_oversampled, cv=5, scoring='roc_auc')
    roc_auc = np.mean(scores)
    # Printing intermediate results
    print(f"Trial {trial.number}, C: {C}, penalty: {penalty}, ROC-AUC: {roc_auc}")
    return roc_auc


study_lr = optuna.create_study(direction="maximize")
study_lr.optimize(objective_function, n_trials=100)

best_params_lr = study_lr.best_params
print("Best Parameters: ", best_params_lr)
print("Best ROC-AUC Score: ", study_lr.best_value)

# Create and save model
best_model_lr = LogisticRegression(**best_params_lr, solver='liblinear', n_jobs=-1)
with open('storage_files/best_models_lr_36_features.pkl', 'wb') as file:
    pickle.dump(best_model_lr, file)

[I 2024-05-13 03:45:22,557] A new study created in memory with name: no-name-9aaf1113-3d34-435d-9f20-74f02769ce02



Logistic Regression started.


[I 2024-05-13 03:45:24,914] Trial 0 finished with value: 0.7025894721285637 and parameters: {'C': 0.29358461003628333, 'penalty': 'l1'}. Best is trial 0 with value: 0.7025894721285637.


Trial 0, C: 0.29358461003628333, penalty: l1, ROC-AUC: 0.7025894721285637


[I 2024-05-13 03:45:30,408] Trial 1 finished with value: 0.7125044730681401 and parameters: {'C': 1.398129321890534, 'penalty': 'l1'}. Best is trial 1 with value: 0.7125044730681401.


...

Trial 97, C: 9.91813883818841, penalty: l1, ROC-AUC: 0.7147356791419883


[I 2024-05-13 03:57:22,013] Trial 98 finished with value: 0.7146822081403255 and parameters: {'C': 8.220663005168845, 'penalty': 'l1'}. Best is trial 74 with value: 0.714758206496837.


Trial 98, C: 8.220663005168845, penalty: l1, ROC-AUC: 0.7146822081403255


[I 2024-05-13 03:57:26,981] Trial 99 finished with value: 0.58144730183221 and parameters: {'C': 7.183908544205467, 'penalty': 'l2'}. Best is trial 74 with value: 0.714758206496837.


Trial 99, C: 7.183908544205467, penalty: l2, ROC-AUC: 0.58144730183221
Best Parameters:  {'C': 9.906911876359292, 'penalty': 'l1'}
Best ROC-AUC Score:  0.714758206496837


In [7]:
# Decision Tree
print('\nDecision Tree started.')
def objective_function(trial):
    max_depth = trial.suggest_int('max_depth', 1, 10)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    criterion = trial.suggest_categorical('criterion', ['gini', 'entropy'])

    model = DecisionTreeClassifier(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        criterion=criterion
    )
    metrics = ['roc_auc','accuracy','precison','recall','f1-score']
    # Using cross_val_score to get the average precision score for each fold
    scores = cross_val_score(model, X_train_oversampled, y_train_oversampled, cv=5, scoring='roc_auc')
    roc_auc = np.mean(scores)
    # Printing intermediate results
    print(f"Trial {trial.number}, max_depth: {max_depth}, min_samples_split: {min_samples_split}, "
          f"min_samples_leaf: {min_samples_leaf}, criterion: {criterion}, ROC-AUC: {roc_auc}")
    return roc_auc

study_dt = optuna.create_study(direction="maximize")
study_dt.optimize(objective_function, n_trials=100)

best_params_dt = study_dt.best_params
print("Best Parameters: ", best_params_dt)
print("Best ROC-AUC Score: ", study_dt.best_value)

# Create and save model
best_model_dt = DecisionTreeClassifier(**best_params_dt)
with open('storage_files/best_models_dt_36_features.pkl', 'wb') as file:
    pickle.dump(best_model_dt, file)

[I 2024-05-13 03:57:27,011] A new study created in memory with name: no-name-33744037-8943-44df-9a1c-dde5afab3132



Decision Tree started.


[I 2024-05-13 03:57:27,479] Trial 0 finished with value: 0.733584655009923 and parameters: {'max_depth': 2, 'min_samples_split': 7, 'min_samples_leaf': 9, 'criterion': 'entropy'}. Best is trial 0 with value: 0.733584655009923.


Trial 0, max_depth: 2, min_samples_split: 7, min_samples_leaf: 9, criterion: entropy, ROC-AUC: 0.733584655009923


[I 2024-05-13 03:57:28,386] Trial 1 finished with value: 0.8779034183423151 and parameters: {'max_depth': 7, 'min_samples_split': 6, 'min_samples_leaf': 7, 'criterion': 'gini'}. Best is trial 1 with value: 0.8779034183423151.


...

Trial 97, max_depth: 10, min_samples_split: 9, min_samples_leaf: 9, criterion: entropy, ROC-AUC: 0.9501313280351434


[I 2024-05-13 03:59:29,847] Trial 98 finished with value: 0.9494186900703333 and parameters: {'max_depth': 9, 'min_samples_split': 10, 'min_samples_leaf': 8, 'criterion': 'entropy'}. Best is trial 52 with value: 0.9515189248568039.


Trial 98, max_depth: 9, min_samples_split: 10, min_samples_leaf: 8, criterion: entropy, ROC-AUC: 0.9494186900703333


[I 2024-05-13 03:59:31,121] Trial 99 finished with value: 0.9503641389356495 and parameters: {'max_depth': 10, 'min_samples_split': 9, 'min_samples_leaf': 9, 'criterion': 'entropy'}. Best is trial 52 with value: 0.9515189248568039.


Trial 99, max_depth: 10, min_samples_split: 9, min_samples_leaf: 9, criterion: entropy, ROC-AUC: 0.9503641389356495
Best Parameters:  {'max_depth': 10, 'min_samples_split': 4, 'min_samples_leaf': 9, 'criterion': 'entropy'}
Best ROC-AUC Score:  0.9515189248568039


In [8]:
# KNN
print('\nKNN started')
def objective_function(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 1, 10)
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])
    p = trial.suggest_int('p', 1, 5)
    metric = trial.suggest_categorical('metric', ['euclidean', 'manhattan', 'minkowski'])

    model = KNeighborsClassifier(
        n_neighbors=n_neighbors,
        weights=weights,
        p=p,
        metric=metric
    )
    metrics = ['roc_auc','accuracy','precison','recall','f1-score']
    # Using cross_val_score to get the average precision score for each fold
    scores = cross_val_score(model, X_train_oversampled, y_train_oversampled, cv=5, scoring='roc_auc')
    roc_auc = np.mean(scores)
    # Printing intermediate results
    print(f"Trial {trial.number}, n_neighbors: {n_neighbors}, weights: {weights}, p: {p}, metric: {metric}, "
          f"ROC-AUC: {roc_auc}")
    return roc_auc


study_knn = optuna.create_study(direction="maximize")
study_knn.optimize(objective_function, n_trials=100)

best_params_knn = study_knn.best_params
print("Best Parameters: ", best_params_knn)
print("Best ROC-AUC Score: ", study_knn.best_value)

# Create and save model
best_model_knn = KNeighborsClassifier(**best_params_knn)
with open('storage_files/best_models_knn_36_features.pkl', 'wb') as file:
    pickle.dump(best_model_knn, file)

[I 2024-05-13 03:59:31,157] A new study created in memory with name: no-name-95ae07e3-52fa-49c7-8bf7-4d50ef4963eb



KNN started


[I 2024-05-13 03:59:31,573] Trial 0 finished with value: 0.8731433487355489 and parameters: {'n_neighbors': 3, 'weights': 'uniform', 'p': 5, 'metric': 'euclidean'}. Best is trial 0 with value: 0.8731433487355489.


Trial 0, n_neighbors: 3, weights: uniform, p: 5, metric: euclidean, ROC-AUC: 0.8731433487355489


[I 2024-05-13 03:59:31,957] Trial 1 finished with value: 0.904412257950335 and parameters: {'n_neighbors': 9, 'weights': 'distance', 'p': 3, 'metric': 'euclidean'}. Best is trial 1 with value: 0.904412257950335.


...

Trial 97, n_neighbors: 10, weights: distance, p: 5, metric: euclidean, ROC-AUC: 0.9039314862432786


[I 2024-05-13 04:08:09,866] Trial 98 finished with value: 0.9355620960016259 and parameters: {'n_neighbors': 7, 'weights': 'uniform', 'p': 4, 'metric': 'manhattan'}. Best is trial 33 with value: 0.9495353969882805.


Trial 98, n_neighbors: 7, weights: uniform, p: 4, metric: manhattan, ROC-AUC: 0.9355620960016259


[I 2024-05-13 04:08:12,052] Trial 99 finished with value: 0.9491534718544801 and parameters: {'n_neighbors': 9, 'weights': 'distance', 'p': 3, 'metric': 'manhattan'}. Best is trial 33 with value: 0.9495353969882805.


Trial 99, n_neighbors: 9, weights: distance, p: 3, metric: manhattan, ROC-AUC: 0.9491534718544801
Best Parameters:  {'n_neighbors': 10, 'weights': 'distance', 'p': 4, 'metric': 'manhattan'}
Best ROC-AUC Score:  0.9495353969882805


In [9]:
# Random Forest
print('\nRandom Forest started')
def objective_function(trial):
    n_estimators = trial.suggest_int('n_estimators', 2, 50)
    max_depth = trial.suggest_int('max_depth', 1, 10)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        n_jobs=-1
    )

    # Using cross_val_score to get the average ROC-AUC score for each fold
    scores = cross_val_score(model, X_train_oversampled, y_train_oversampled, cv=5, scoring='roc_auc')
    roc_auc = np.mean(scores)
    # Printing intermediate results
    print(f"Trial {trial.number}, n_estimators: {n_estimators}, max_depth: {max_depth}, "
          f"min_samples_split: {min_samples_split}, min_samples_leaf: {min_samples_leaf}, ROC-AUC: {roc_auc}")
    return roc_auc


study_rf = optuna.create_study(direction="maximize")
study_rf.optimize(objective_function, n_trials=100)

best_params_rf = study_rf.best_params
print("Best Parameters: ", best_params_rf)
print("Best ROC-AUC: Score: ", study_rf.best_value)

# Create and save model
best_model_rf = RandomForestClassifier(**best_params_rf, n_jobs=-1)
with open('storage_files/best_models_rf_36_features.pkl', 'wb') as file:
    pickle.dump(best_model_rf, file)

label_encoder = LabelEncoder()
# Fit the encoder and transform the target variable
y_train_oversampled_encoded = label_encoder.fit_transform(y_train_oversampled)
# This suppresses printing logs
optuna.logging.set_verbosity(optuna.logging.WARNING)

[I 2024-05-13 04:08:12,078] A new study created in memory with name: no-name-fdfe4a8f-02e1-4a69-8e99-8fe79e399ac2



Random Forest started


[I 2024-05-13 04:08:12,561] Trial 0 finished with value: 0.9294750917337827 and parameters: {'n_estimators': 3, 'max_depth': 8, 'min_samples_split': 9, 'min_samples_leaf': 10}. Best is trial 0 with value: 0.9294750917337827.


Trial 0, n_estimators: 3, max_depth: 8, min_samples_split: 9, min_samples_leaf: 10, ROC-AUC: 0.9294750917337827


[I 2024-05-13 04:08:13,271] Trial 1 finished with value: 0.8686899035696012 and parameters: {'n_estimators': 15, 'max_depth': 2, 'min_samples_split': 6, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.9294750917337827.


...

Trial 97, n_estimators: 41, max_depth: 10, min_samples_split: 5, min_samples_leaf: 1, ROC-AUC: 0.9819112464965365


[I 2024-05-13 04:14:42,012] Trial 98 finished with value: 0.9578476782454327 and parameters: {'n_estimators': 40, 'max_depth': 6, 'min_samples_split': 6, 'min_samples_leaf': 1}. Best is trial 56 with value: 0.9844000197076754.


Trial 98, n_estimators: 40, max_depth: 6, min_samples_split: 6, min_samples_leaf: 1, ROC-AUC: 0.9578476782454327


[I 2024-05-13 04:14:45,880] Trial 99 finished with value: 0.9797870874449568 and parameters: {'n_estimators': 31, 'max_depth': 9, 'min_samples_split': 6, 'min_samples_leaf': 1}. Best is trial 56 with value: 0.9844000197076754.


Trial 99, n_estimators: 31, max_depth: 9, min_samples_split: 6, min_samples_leaf: 1, ROC-AUC: 0.9797870874449568
Best Parameters:  {'n_estimators': 48, 'max_depth': 10, 'min_samples_split': 8, 'min_samples_leaf': 4}
Best ROC-AUC: Score:  0.9844000197076754


In [10]:
# XGBoost
print('\nXGBoost started')
def objective_function(trial):
    n_estimators = trial.suggest_int('n_estimators', 2, 50)
    max_depth = trial.suggest_int('max_depth', 1, 10)
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.9, log=True)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 10)
    subsample = trial.suggest_float('subsample', 0.5, 1.0)
    colsample_bytree = trial.suggest_float('colsample_bytree', 0.5, 1.0)
    gamma = trial.suggest_float('gamma', 0, 1.0)
    reg_alpha = trial.suggest_float('reg_alpha', 0, 1)
    reg_lambda = trial.suggest_float('reg_lambda', 0, 1)

    model = XGBClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        min_child_weight=min_child_weight,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        gamma=gamma,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        use_label_encoder=False,
        n_jobs=-1
    )

    # Using cross_val_score to get the average ROC-AUC score for each fold
    scores = cross_val_score(model, X_train_oversampled, y_train_oversampled_encoded, cv=5, scoring='roc_auc')
    roc_auc = np.mean(scores)
    # Printing intermediate results
    print(f"Trial {trial.number}, n_estimators: {n_estimators}, max_depth: {max_depth}, learning_rate: {learning_rate},"
          f"min_child_weight: {min_child_weight}, subsample: {subsample}, colsample_bytree: {colsample_bytree}, "
          f"gamma: {gamma}, reg_alpha: {reg_alpha}, reg_lambda: {reg_lambda}, ROC-AUC: {roc_auc}")
    return roc_auc


study_xgb = optuna.create_study(direction="maximize")
study_xgb.optimize(objective_function, n_trials=100)

best_params_xgb = study_xgb.best_params
print("Best Parameters: ", best_params_xgb)
print("Best ROC-AUC Score: ", study_xgb.best_value)

best_model_xgb = XGBClassifier(**best_params_xgb, use_label_encoder=False, n_jobs=-1)
with open('storage_files/best_models_xgb_36_features.pkl', 'wb') as file:
    pickle.dump(best_model_xgb, file)


XGBoost started
Trial 0, n_estimators: 3, max_depth: 2, learning_rate: 0.0030813089476601644,min_child_weight: 7, subsample: 0.5311549574061932, colsample_bytree: 0.9904951786289279, gamma: 0.16263061376276244, reg_alpha: 0.17079289788979046, reg_lambda: 0.044208855112493506, ROC-AUC: 0.7811500536303772
Trial 1, n_estimators: 42, max_depth: 6, learning_rate: 0.23544204420936748,min_child_weight: 4, subsample: 0.8129557133097995, colsample_bytree: 0.9346641825707578, gamma: 0.9414249643904902, reg_alpha: 0.09842774179324987, reg_lambda: 0.6936135940781807, ROC-AUC: 0.9875758294818295
Trial 2, n_estimators: 18, max_depth: 6, learning_rate: 0.03898530000013839,min_child_weight: 2, subsample: 0.666965501471807, colsample_bytree: 0.9470189253050632, gamma: 0.4301248503188094, reg_alpha: 0.15748589606396823, reg_lambda: 0.041994804639860606, ROC-AUC: 0.9483920401889658
Trial 3, n_estimators: 41, max_depth: 6, learning_rate: 0.0018380184708672758,min_child_weight: 10, subsample: 0.7601484948

In [11]:
# Model selection - Compare Performance
with open('storage_files/best_models_lr_36_features.pkl', 'rb') as file:
    best_model_lr = pickle.load(file)
with open('storage_files/best_models_dt_36_features.pkl', 'rb') as file:
    best_model_dt = pickle.load(file)
with open('storage_files/best_models_knn_36_features.pkl', 'rb') as file:
    best_model_knn = pickle.load(file)
with open('storage_files/best_models_rf_36_features.pkl', 'rb') as file:
    best_model_rf = pickle.load(file)
with open('storage_files/best_models_xgb_36_features.pkl', 'rb') as file:
    best_model_xgb = pickle.load(file)

print("\nTesting Performances...Please wait")
best_model_lr.fit(X_train_oversampled, y_train_oversampled)
predicted_probs = best_model_lr.predict_proba(X_test_transformed)[:, 1]
lr_performance = roc_auc_score(y_test, predicted_probs)

best_model_dt.fit(X_train_oversampled, y_train_oversampled)
predicted_probs = best_model_dt.predict_proba(X_test_transformed)[:, 1]
dt_performance = roc_auc_score(y_test, predicted_probs)

best_model_knn.fit(X_train_oversampled, y_train_oversampled)
predicted_probs = best_model_knn.predict_proba(X_test_transformed)[:, 1]
knn_performance = roc_auc_score(y_test, predicted_probs)

best_model_rf.fit(X_train_oversampled, y_train_oversampled)
predicted_probs = best_model_rf.predict_proba(X_test_transformed)[:, 1]
rf_performance = roc_auc_score(y_test, predicted_probs)

best_model_xgb.fit(X_train_oversampled, y_train_oversampled_encoded)
predicted_probs = best_model_xgb.predict_proba(X_test_transformed)[:, 1]
xgb_performance = roc_auc_score(y_test, predicted_probs)

# Test performance of the models are
print(f"Logistic Regression Test ROCAUC: {lr_performance}")
print(f"Decision Tree Test ROCAUC: {dt_performance}")
print(f"KNN Test ROCAUC: {knn_performance}")
print(f"Random Forest Test ROCAUC: {rf_performance}")
print(f"XGBoost Test ROCAUC: {xgb_performance}")


Testing Performances...Please wait
Logistic Regression Test ROCAUC: 0.7281135531135531
Decision Tree Test ROCAUC: 0.46451465201465203
KNN Test ROCAUC: 0.49203296703296706
Random Forest Test ROCAUC: 0.8086080586080586
XGBoost Test ROCAUC: 0.7736263736263735
